In [501]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup
import re

In [502]:
site = 'https://www.hanbit.co.kr/'
url = f'{site}store/books/new_book_list.html'
req = requests.get(url)
html = req.text
#html

In [503]:
soup = BeautifulSoup(html, 'html.parser')

In [504]:
lis = soup.select('div.sub_book_list_area>li')
len(lis)

20

In [505]:
li = lis[1]
href = li.find('a')['href']
href

'/store/books/look.php?p_code=B6972407557'

In [506]:
#제목
book_tit = li.select_one('.book_tit').get_text()
book_tit

'다시, 배우다'

In [507]:
#저자
book_writer = li.select_one('.book_writer').get_text()
book_writer

'폴 김 '

In [508]:
sub_url = site + href
sub_url

'https://www.hanbit.co.kr//store/books/look.php?p_code=B6972407557'

In [509]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [510]:
#역자
trans= sub_soup.select('.info_list>li>span')
trans= trans[-5].get_text()
trans

'폴 김 '

In [587]:
#출간일
day = sub_soup.select_one('.info_list').get_text().split('\n')
day = day[1:][1]
day = re.sub('[^0-9]','',day)
day


'20211102'

In [512]:
#페이지
page= sub_soup.select('.info_list>li>span')[-3].get_text()
page = int(re.sub('[^0-9]','',page))
page

304

In [513]:
#정가
mon = sub_soup.select('.payment_box.curr>p')[0].get_text()
mon = int(re.sub('[^0-9]','',mon))
mon

16000

In [514]:
#할인가
disc = sub_soup.select('.payment_box.curr>p')[1].get_text()
disc = int(re.sub('[^0-9]','',disc))
disc

1440010

In [515]:
mil = sub_soup.select('.payment_box.curr>p')[2].get_text()
mil = int(re.sub('[^0-9]','',mil))
mil

8005

In [521]:
lines = []
for li in lis:
    href = li.find('a')['href']
    book_tit = li.select_one('.book_tit').get_text().strip()
    book_writer = li.select_one('.book_writer').get_text().strip()

    sub_url = site + href
    sub_req = requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html,'html.parser')
    trans= sub_soup.select('.info_list>li>span')[-5].get_text().strip()
    day = sub_soup.select('.info_list>li>span')[-4].get_text().strip()
    day = int(re.sub('[^0-9]','',day))
    page= sub_soup.select('.info_list>li>span')[-3].get_text()
    page = re.sub('[^0-9]','',page)
    mon = sub_soup.select('.payment_box.curr>p')[0].get_text().strip()
    mon = re.sub('[^0-9]','',mon)
    disc = sub_soup.select('.payment_box.curr>p')[1].get_text().strip()
    disc = re.sub('[^0-9]','',disc)
    mil = sub_soup.select('.payment_box.curr>p')[2].get_text().strip()
    mil = re.sub('[^0-9]','',mil)
    lines.append([book_tit, book_writer, trans, day, page, mon, disc, mil])

df = pd.DataFrame(lines, columns=['제목', '저자', '역자', '출간일', '페이지', '정가', '할인가', '마일리지'])
df

,제목,저자,역자,출간일,페이지,정가,할인가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,"허영은 , 유우종 (감수)",20211214,416,19800,1782010,9905
1,"다시, 배우다",폴 김,폴 김,20211210,304,16000,1440010,8005
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,"김민성 , 곽서희",20211210,376,22000,1980010,11005
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영","박재연 , 공인영",20211201,384,18000,1620010,9005
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale","나양 , 이승배 , 임윤철",20211130,600,36000,360000,10803
5,엑셀 매크로&VBA 업무 공략집,최준선,최준선,20211130,328,22000,1980010,11005
6,데이터 과학자 되는 법,"에밀리 로빈슨 , 재클린 놀리스",이창화,20211126,372,24000,2160010,12005
7,돈 되는 기획,김도균,김도균,20211126,240,14500,1305010,7255
8,빠르게 찾아 바로 적용하는 엑셀 에센스 사전 100,한은숙,한은숙,20211125,300,18000,1620010,9005
9,초등 어휘력이 공부력이다,박명선,박명선,20211125,412,18000,1620010,9005
